In [1]:
# Librerías necesarias
import nltk
import tqdm

nltk.data.path.append('.')

# Limpieza y tokenización

In [2]:
# Actualiza esta linea con la ruta del txt
PATH_TEXT = 'D:/01_NLP/NLP_especialization/02_Probabilistic_models/03_Autocomplet/01_resumen/En_US_twitter.txt'

# Leyendo los textos
with open(PATH_TEXT, "r", encoding="utf8") as f:
    data = f.read()

In [3]:
def tokeniza_frase(frase):
    ''' convierte una frase en tokens '''
    frase = frase.lower()
    frase = frase.strip()
    token = nltk.word_tokenize(frase)
    return token

In [4]:
tokeniza_frase('This is an example')

['this', 'is', 'an', 'example']

In [5]:
def tokeniza_data(data):
    # Paso 1: dividiendo por saltos de línea (\n) para generar frases
    frases = data.split('\n')
    # Paso 2: Tokenizando cada frase
    frases = [tokeniza_frase(x) for x in frases]
    # Paso 3: Manteniendo textos que no sean vacios
    frases = [x for x in frases if len(x) > 0]
    return frases

In [6]:
frases_tokenizadas = tokeniza_data(data)

In [7]:
print("frases tokenizadas", frases_tokenizadas[3], frases_tokenizadas[8])

frases tokenizadas ['so', 'tired', 'd', ';', 'played', 'lazer', 'tag', '&', 'ran', 'a', 'lot', 'd', ';', 'ughh', 'going', 'to', 'sleep', 'like', 'in', '5', 'minutes', ';', ')'] ['the', 'new', 'sundrop', 'commercial', '...', 'hehe', 'love', 'at', 'first', 'sight']


# Procesamiento de datos

In [8]:
def calcula_conteo_tokens(frases_tokenizadas):
    ''' generamos un diccionario con cantidad de veces que aparece cada token '''
    conteo_token_diccionario = {}
    for frase in frases_tokenizadas:
        for token in frase:
            if token not in conteo_token_diccionario.keys():
                conteo_token_diccionario[token] = 1
            else:
                conteo_token_diccionario[token] += 1
    return conteo_token_diccionario

In [9]:
def genera_vocabulario(conteo_token_diccionario, umbral_minimo):
    ''' devuelve lista de palabras (vocabulario) que superan el umbral '''
    vocabulario_valido = [x for x, v in conteo_token_diccionario.items() if v >= umbral_minimo]
    return vocabulario_valido

In [10]:
def limpieza_frases_tokenizadas(frases_tokenizadas, vocabulario, token_missing="<unk>"):
    ''' reemplaza cada palabra por el valor de missing '''
    vocabulario = set(vocabulario)
    frases_tokenizadas_sin_missing = []
    for frase in frases_tokenizadas:
        frase_reemplazado = []
        for token in frase:
            if token in vocabulario:
                # Si el token está en el vocabulario, lo mantenemos
                frase_reemplazado.append(token)
            else:
                # Si el token no está en el vocabulario, lo reemplazamos por el token_missing
                frase_reemplazado.append(token_missing)
        frases_tokenizadas_sin_missing.append(frase_reemplazado)
    return frases_tokenizadas_sin_missing

In [11]:
def preprocesa_data(frases_tokenizadas, umbral_minimo, token_missing="<unk>"):
    ''' Calculamos la cantidad de veces que aparece nuestro token '''
    conteo_palabra_diccionario = calcula_conteo_tokens(frases_tokenizadas)
    ''' generamos nuestro vocabulario '''
    vocabulario = genera_vocabulario(conteo_palabra_diccionario, umbral_minimo)
    ''' Limpiamos nuestra lista de tokens usando el vocabulario '''
    frases_tokenizadas_sin_missing = limpieza_frases_tokenizadas(frases_tokenizadas, vocabulario, token_missing)
    return frases_tokenizadas_sin_missing, vocabulario

In [12]:
frases_tokenizadas, vocabulario = preprocesa_data(frases_tokenizadas, umbral_minimo=2, token_missing="<unk>")

In [13]:
print("cantidad en el vocabulario", len(vocabulario))
print("Ejemplos:", vocabulario[:5])
print("frases tokenizadas", frases_tokenizadas[3], frases_tokenizadas[8])

cantidad en el vocabulario 17031
Ejemplos: ['how', 'are', 'you', '?', 'btw']
frases tokenizadas ['so', 'tired', 'd', ';', 'played', '<unk>', 'tag', '&', 'ran', 'a', 'lot', 'd', ';', '<unk>', 'going', 'to', 'sleep', 'like', 'in', '5', 'minutes', ';', ')'] ['the', 'new', '<unk>', 'commercial', '...', 'hehe', 'love', 'at', 'first', 'sight']


# Desarrollando modelo de lenguaje basado en n-grams

## Conteo n-grams

In [14]:
def conteo_n_grams(frases_tokenizadas, n, token_inicio="<s>", token_final="<e>"):
    ''' calcula un diccionario con la cantidad que veces que aparece los n_grams '''
    diccionario_n_grams = {}
    for frase in frases_tokenizadas:
        ''' agregamos token inicial y final a la frase '''
        nueva_frase = [token_inicio] * n + frase + [token_final]
        for i in range(len(nueva_frase) - n + 1):
            ''' generamos nuestro n-gram '''
            n_gram = nueva_frase[i : (i + n)]
            n_gram = tuple(n_gram)
            if n_gram not in diccionario_n_grams.keys():
                ''' inicializamos nuestro n-gram si no está en nuestro diccionario '''
                diccionario_n_grams[n_gram] = 1
            else:
                diccionario_n_grams[n_gram] += 1
    return diccionario_n_grams

## Estima probabilidad condicional

In [15]:
def estima_probabilidad(palabra, n_gram_previo, diccionario_n_gram, diccionario_n1_gram, 
                        vocabulario, k=1.0, token_inicial='<s>', token_missing='<unk>'):
    ''' calcula la probabilidad de una palabra dado el n_gram anterior '''
    ''' verificamos si la palabra está en nuestro diccionario sino lo reemplazamos por missing '''
    n_gram_previo = [x if x in vocabulario else x if x == token_inicial else token_missing for x in n_gram_previo]
    n_gram_previo = tuple(n_gram_previo)
    tamano_vocabulario = len(vocabulario)
    ''' cantidad de veces que aparece el n_gram '''
    cantidad_previo_n_gram = diccionario_n_gram.get(n_gram_previo, 0)
    ''' Calculamos el denominador '''
    denominador = cantidad_previo_n_gram + k * tamano_vocabulario
    ''' generando el n_gram + 1 '''
    n_gram_1 = n_gram_previo + (palabra,)
    ''' cantidad de veces que aparece el n_gram_1 '''
    cantidad_n_gram_1 = diccionario_n1_gram.get(n_gram_1, 0)
    ''' Calculamos el numerador '''
    numerador = cantidad_n_gram_1 + k
    ''' calculando la probabilidad '''
    probabilidad = numerador / denominador
    return probabilidad

In [16]:
def estima_probabilidades(n_gram_previo, diccionario_n_gram, diccionario_n1_gram, vocabulario, k=1.0,
                          token_inicial='<s>', token_final='<e>', token_missing='<unk>'):
    n_gram_previo = tuple(n_gram_previo)
    ''' actualizamos nuestro vocabulario con el token final y token missin '''
    vocabulary_nuevo = vocabulario + [token_final, token_missing]
    vocabulary_nuevo = set(vocabulary_nuevo)
    probabilidades = {}
    for palabra in vocabulary_nuevo:
        ''' calculamos la probabilidad para cada palabra de nuestro vocabulario '''
        probabilidad = estima_probabilidad(palabra, n_gram_previo,
                                           diccionario_n_gram, diccionario_n1_gram,
                                           vocabulary_nuevo, 
                                           k,
                                           token_inicial,
                                           token_missing
                                          )
        probabilidades[palabra] = probabilidad
    return probabilidades 

# Sugerencias

In [17]:
def sugiere_siguiente_palabra(token_previo, diccionario_n_gram, diccionario_n1_gram, vocabulario, k=1, empieza_con=None, 
                              token_inicial='<s>', token_final='<e>', token_missing='<unk>'):
    ''' calculamos el tamaño del n-gram '''
    n = len(list(diccionario_n_gram.keys())[0])
    
    if len(token_previo) >= n:
        ''' si el token es mayor a n nos quedamos con los n últimos tokens '''
        n_gram_previo = token_previo[-n:]
    else:
        ''' caso contrario, agregamos el token_inicial '''
        n_gram_previo = [token_inicial] * (n - len(token_previo)) + token_previo
        
    ''' calculamos las probabilidades condicionales para nuestro vocabulario '''
    probabilidades = estima_probabilidades(n_gram_previo, 
                                           diccionario_n_gram,
                                           diccionario_n1_gram, 
                                           vocabulario, 
                                           k,
                                           token_final,
                                           token_missing
                                          )
    ''' empezamos a sugerir palabras '''
    sugerencia = None
    maxima_probabilidad = 0
    
    for palabra, probabilidad in probabilidades.items():
        if empieza_con is not None:
            ''' verificamos si la palabra empieza con empieza_con '''
            if len(palabra) < len(empieza_con) or empieza_con != palabra[:len(empieza_con)]:
                continue
        if probabilidad > maxima_probabilidad:
            ''' nos quedamos con la palabra que maximice la probabilidad '''
            sugerencia = palabra
            maxima_probabilidad = probabilidad
    return sugerencia, maxima_probabilidad

In [18]:
def devuelve_sugerencias_token_previo(token_previo, lista_diccionario_n_gram, vocabulario, k=1.0, empieza_con=None,
                                     token_inicial='<s>', token_final='<e>', token_missing='<unk>'):
    ''' devuelve sugerencia para una lista de diccionarios con los n-gram '''
    ''' contamos la lista de modelos sugeridos '''
    cantidad_modelos = len(lista_diccionario_n_gram)
    sugerencias = []
    for i in range(cantidad_modelos-1):
        ''' calculamos las sugerencias para cada diccionario n-gram '''
        diccionario_n_gram = lista_diccionario_n_gram[i]
        diccionario_n1_gram = lista_diccionario_n_gram[i+1]
        sugerencia  = sugiere_siguiente_palabra(token_previo, diccionario_n_gram,
                                                diccionario_n1_gram, vocabulario,
                                                k, empieza_con,
                                                token_inicial,
                                                token_final,
                                                token_missing
                                               )
        ''' guardamos la sugerencia en la lista de sugerencias '''
        sugerencias.append(sugerencia)
    return sugerencias

In [19]:
def devuelve_sugerencias(texto, lista_diccionario_n_gram, vocabulario, k=1.0, empieza_con=None,
                        token_inicial='<s>', token_final='<e>', token_missing='<unk>'):
    ''' tokenizamos nuestro texto '''
    token_texto = tokeniza_frase(texto)
    ''' calculamos la sugerencias para una lista de diccionarios n-grams '''
    sugerencias = devuelve_sugerencias_token_previo(token_texto, lista_diccionario_n_gram, vocabulario, 
                                                    k, 
                                                    empieza_con,
                                                    token_inicial,
                                                    token_final,
                                                    token_missing)
    ''' devolvemos la salida '''
    return sugerencias

# Probando el algoritmo

In [20]:
''' generamos nuestra lista de diccionarios con los conteos n-grams '''
lista_diccionario_n_gram = [conteo_n_grams(frases_tokenizadas, n) for n in range(1, 10)]

In [21]:
texto = "i like"
sugerencias = devuelve_sugerencias(texto, lista_diccionario_n_gram, vocabulario, 1.0)

In [22]:
sugerencias

[('a', 0.01583652618135377),
 ('the', 0.0015098722415795587),
 ('you', 0.00011740534194305841),
 ('stroll', 5.871301080319399e-05),
 ('stroll', 5.871301080319399e-05),
 ('stroll', 5.871301080319399e-05),
 ('stroll', 5.871301080319399e-05),
 ('stroll', 5.871301080319399e-05)]

In [23]:
texto = "i like a"
sugerencias = devuelve_sugerencias(texto, lista_diccionario_n_gram, vocabulario, 1.0, 'i')

In [24]:
sugerencias

[('in', 0.00010207206287639073),
 ('inquiry', 5.7666801222536187e-05),
 ('inquiry', 5.870956378794105e-05),
 ('inquiry', 5.871301080319399e-05),
 ('inquiry', 5.871301080319399e-05),
 ('inquiry', 5.871301080319399e-05),
 ('inquiry', 5.871301080319399e-05),
 ('inquiry', 5.871301080319399e-05)]

In [25]:
texto = "this is an"
sugerencias = devuelve_sugerencias(texto, lista_diccionario_n_gram, vocabulario, 1.0)

In [26]:
sugerencias

[('<unk>', 0.007158739627962471),
 ('<unk>', 0.0004679457182966776),
 ('old', 0.00011740534194305841),
 ('stroll', 5.871301080319399e-05),
 ('stroll', 5.871301080319399e-05),
 ('stroll', 5.871301080319399e-05),
 ('stroll', 5.871301080319399e-05),
 ('stroll', 5.871301080319399e-05)]

In [27]:
texto = "i want to go to"
sugerencias = devuelve_sugerencias(texto, lista_diccionario_n_gram, vocabulario, 1.0)

In [28]:
sugerencias

[('be', 0.028768208691412202),
 ('the', 0.002425362360686031),
 ('the', 0.0011099427503212993),
 ('the', 0.00035198873636043646),
 ('the', 0.00017608733932030288),
 ('stroll', 5.871301080319399e-05),
 ('stroll', 5.871301080319399e-05),
 ('stroll', 5.871301080319399e-05)]

In [29]:
texto = "how are you"
sugerencias = devuelve_sugerencias(texto, lista_diccionario_n_gram, vocabulario, 1.0)

In [30]:
sugerencias

[("'re", 0.025338933852405834),
 ('?', 0.003025632242963978),
 ('?', 0.0017544885665828412),
 ('my', 0.0001174191275758821),
 ('stroll', 5.871301080319399e-05),
 ('stroll', 5.871301080319399e-05),
 ('stroll', 5.871301080319399e-05),
 ('stroll', 5.871301080319399e-05)]

In [31]:
texto = "how are"
sugerencias = devuelve_sugerencias(texto, lista_diccionario_n_gram, vocabulario, 1.0)

In [32]:
sugerencias

[('you', 0.023831706958269996),
 ('you', 0.0039731229915279),
 ('you', 0.0001174191275758821),
 ('stroll', 5.871301080319399e-05),
 ('stroll', 5.871301080319399e-05),
 ('stroll', 5.871301080319399e-05),
 ('stroll', 5.871301080319399e-05),
 ('stroll', 5.871301080319399e-05)]

In [33]:
texto = "i am very"
sugerencias = devuelve_sugerencias(texto, lista_diccionario_n_gram, vocabulario, 1.0)

In [34]:
sugerencias

[('cool', 0.0014271850202660272),
 ('confused', 0.00011737089201877934),
 ('confused', 0.00011739155954686858),
 ('stroll', 5.871301080319399e-05),
 ('stroll', 5.871301080319399e-05),
 ('stroll', 5.871301080319399e-05),
 ('stroll', 5.871301080319399e-05),
 ('stroll', 5.871301080319399e-05)]

In [35]:
estima_probabilidad('?', ['how', 'are', 'you'], lista_diccionario_n_gram[2], lista_diccionario_n_gram[3],
                    vocabulario, k=1.0, token_inicial='<s>', token_missing='<unk>')

0.0017545911802550006

In [36]:
estima_probabilidad('cool', ['how', 'are', 'you'], lista_diccionario_n_gram[2], lista_diccionario_n_gram[3],
                    vocabulario, k=1.0, token_inicial='<s>', token_missing='<unk>')

5.848637267516669e-05